# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [4]:
import main
import main

import psycopg2 as pg
import pandas as pd
#ZAD1
connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')
film_same_length=pd.read_sql('SELECT length, count(title) from film GROUP BY length ORDER BY length',con=connection)
print(film_same_length)

     length  count
0        46      5
1        47      7
2        48     11
3        49      5
4        50      9
..      ...    ...
135     181     10
136     182      6
137     183      5
138     184      8
139     185     10

[140 rows x 2 columns]


In [5]:
#ZAD2
same_country = pd.read_sql('SELECT city.city, COUNT(customer.last_name) \
                            FROM customer \
                            INNER JOIN address ON customer.address_id = address.address_id \
                            INNER JOIN city ON address.city_id = city.city_id \
                            GROUP BY city.city \
                            ORDER BY COUNT(customer.last_name) DESC', con=connection)
print(same_country)

                     city  count
0                  London      2
1                  Aurora      2
2               Molodetno      1
3                 Yerevan      1
4    Vitria de Santo Anto      1
..                    ...    ...
592          Garden Grove      1
593          Novoterkassk      1
594               Asuncin      1
595              Maracabo      1
596            Rae Bareli      1

[597 rows x 2 columns]


In [6]:
#ZAD3
avg_cost = pd.read_sql("SELECT AVG(payment.amount) FROM payment", con=connection)
print(avg_cost)

        avg
0  4.200606


In [7]:
#ZAD4
number_in_categories = pd.read_sql("SELECT category.name, COUNT(category.name) \
                                    FROM film \
                                    INNER JOIN film_category \
                                    ON film.film_id = film_category.film_id \
                                    INNER JOIN category \
                                    ON film_category.category_id = category.category_id \
                                    GROUP BY category.name \
                                    ORDER BY category.name", con=connection)

print(number_in_categories)

           name  count
0        Action     64
1     Animation     66
2      Children     60
3      Classics     57
4        Comedy     58
5   Documentary     68
6         Drama     62
7        Family     69
8       Foreign     73
9         Games     61
10       Horror     56
11        Music     51
12          New     63
13       Sci-Fi     61
14       Sports     74
15       Travel     57


In [8]:
#ZAD5
customers= pd.read_sql("SELECT country.country, count(customer.last_name) \
                        FROM customer \
                        INNER JOIN address ON customer.address_id = address.address_id \
                        INNER JOIN city ON address.city_id = city.city_id \
                        INNER JOIN country ON city.country_id = country.country_id \
                        GROUP BY country.country \
                        ORDER BY count(customer.last_name) DESC",con=connection)
print(customers)

           country  count
0            India     60
1            China     53
2    United States     36
3            Japan     31
4           Mexico     30
..             ...    ...
103        Hungary      1
104    New Zealand      1
105        Senegal      1
106      Greenland      1
107          Tonga      1

[108 rows x 2 columns]


In [9]:
#ZAD6
store = pd.read_sql("SELECT store.store_id, COUNT(customer.customer_id) \
                    FROM customer \
                    INNER JOIN store \
                    ON customer.store_id = store.store_id \
                    GROUP BY store.store_id \
                    HAVING COUNT(*) BETWEEN 100 AND 300", con=connection)

print(store)

   store_id  count
0         2    273


In [10]:
#ZAD7
over_200= pd.read_sql("SELECT customer.first_name, customer.last_name, SUM(film.length) as watched \
                        FROM customer \
                        INNER JOIN rental ON customer.customer_id = rental.customer_id \
                        INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id \
                        INNER JOIN film ON inventory.film_id = film.film_id \
                        GROUP BY customer.customer_id \
                        HAVING SUM(film.length) > 200*60",con=connection)
print(over_200)

Empty DataFrame
Columns: [first_name, last_name, watched]
Index: []


In [11]:
#ZAD8
avg_cost = pd.read_sql("SELECT AVG(payment.amount) FROM payment", con=connection)
print(avg_cost)

        avg
0  4.200606


In [12]:
#ZAD9
length_in_categories = pd.read_sql("SELECT category.name, AVG(film.length) \
                                    FROM film \
                                    INNER JOIN film_category \
                                    ON film.film_id = film_category.film_id \
                                    INNER JOIN category \
                                    ON film_category.category_id = category.category_id \
                                    GROUP BY category.name \
                                    ORDER BY AVG(film.length)", con=connection)

print(length_in_categories)

           name         avg
0        Sci-Fi  108.196721
1   Documentary  108.750000
2      Children  109.800000
3     Animation  111.015152
4           New  111.126984
5        Action  111.609375
6      Classics  111.666667
7        Horror  112.482143
8        Travel  113.315789
9         Music  113.647059
10       Family  114.782609
11       Comedy  115.827586
12        Drama  120.838710
13      Foreign  121.698630
14        Games  127.836066
15       Sports  128.202703


In [13]:
#ZAD10
length_in_categories = pd.read_sql("SELECT category.name, MAX(LENGTH(film.title)) \
                                    FROM film \
                                    INNER JOIN film_category \
                                    ON film.film_id = film_category.film_id \
                                    INNER JOIN category \
                                    ON film_category.category_id = category.category_id \
                                    GROUP BY category.name \
                                    ORDER BY MAX(LENGTH(film.title))", con=connection)

print(length_in_categories)

           name  max
0        Sci-Fi   20
1      Children   20
2       Foreign   20
3         Games   21
4           New   21
5         Music   22
6   Documentary   22
7        Family   22
8     Animation   22
9         Drama   22
10       Travel   23
11       Comedy   23
12     Classics   23
13       Action   23
14       Sports   25
15       Horror   27


In [14]:
#ZAD11
length_of_films_in_categories = pd.read_sql("SELECT category.name, MAX(film.length) \
                                    FROM film \
                                    INNER JOIN film_category \
                                    ON film.film_id = film_category.film_id \
                                    INNER JOIN category \
                                    ON film_category.category_id = category.category_id \
                                    GROUP BY category.name \
                                    ORDER BY MAX(film.length) DESC", con=connection)

print(length_of_films_in_categories)

           name  max
0     Animation  185
1        Sci-Fi  185
2        Comedy  185
3         Games  185
4        Travel  185
5         Music  185
6        Action  185
7        Family  184
8        Sports  184
9       Foreign  184
10     Classics  184
11          New  183
12  Documentary  183
13       Horror  181
14        Drama  181
15     Children  178
